In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [2]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
class Website:
# A Utility class to represent a website that we have scraped
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [7]:
ed = Website("https://www.tomshardware.com/pc-components/cpus/intels-fastest-gaming-chip-hits-all-time-low-for-black-friday-core-i9-14900k-drops-to-usd413")
print(ed.title)
print(ed.text)

Intel's fastest gaming chip hits all-time low for Cyber Monday — Core i9-14900K drops to $413 | Tom's Hardware
Skip to main content
Open menu
Close menu
Tom's Hardware
Tom's Hardware
Search
Search Tom's Hardware
RSS
US Edition
UK
US
Australia
Canada
Best Picks
Raspberry Pi
CPUs
GPUs
3D Printers
News
Coupons
More
Newsletter
Reviews
PC Components
Motherboards
SSDs
PC Building
Monitors
Laptops
Gaming
Cooling
RAM
Power Supplies
Cases
3D Printers
Desktops
Overclocking
Peripherals
About Us
Forums
Trending
Black Friday SSD Deals Live
15 under-$50 gadgets
Black Friday Monitor Deals Live
Best Black Friday Tech Deals
Core Ultra 9 285K
PC Components
CPUs
Intel's fastest gaming chip hits all-time low for Cyber Monday — Core i9-14900K drops to $413
Deals
By
Roshan Ashraf Shaikh
published
30 November 2024
Intel's flagship Raptor Lake Refresh CPU drops to a very attractive price.
Comments
(1)
When you purchase through links on our site, we may earn an affiliate commission.
Here’s how it works
.
(Imag